## Finalizing Bldg Permit Data for Creating Visualizations
- Discovered when starting to create visualizations that I needed one cleaned dataset with **both** types of permits Applications and Permits; and I only needed *some* of the cleaning steps used in the prior notebooks, and in a different sequence than they had been applied. I may need those steps later, so I'm leaving those notebooks intact.
- SOLUTION: Creating new notebook 3_finalize_data and will only do the steps needed to get final, single dataset.
- NAMING: To differentiatl these two bldg. permit dfs from prior notebooks, using these names:
    - df1_applic (permit applications)  
    - df2_issued (permits issued)  
    - df3_all (final, merged file)  
- PREPARE
    1. Column name cleanup
    2. Data types cleanup
    3. New column: 'category' (Residential, Commercial, Other)
    4. New column: 'type_subtype' concatenation
    5. Drop addresses with "0" house number
    6. Pull lat/lon from mapped_location
    7. Drop rows with null lat/lon
    8. Sort by address (prerequsite to dropping duplicates)
    9. Drop duplicates:  Unable to do this. Will come back to it after internal walkthrough.
    10. Convert to geodataframe
    11. Keep only addresses within Davidson County
    12. Add Neighborhood Association names
    13. Merge
    14. Export

In [1]:
import pandas as pd
import numpy as np
import geopandas as gpd   # Prerequisite: Anotes_and_docsctivate geospatial environment via Conda Prompt
import matplotlib.pyplot as plt  
import folium                   
from folium.plugins import MarkerCluster
import datetime
from shapely.geometry import Point  
import re # To use RegEx to pull ot lat/long from building permit applications/issued

## Reading in raw data for building permits

In [2]:
# Building Dept. Permit Applications

df1_applic = pd.read_csv('../data/raw/Building_Permit_Applications_2020_06_05.csv')
df1_applic.head(2)

,Permit #,Permit Type Description,Permit Subtype Description,Parcel,Date Entered,Date Issued,Construction Cost,Address,City,State,ZIP,Subdivision / Lot,Contact,Permit Type,Permit Subtype,IVR Tracking #,Purpose,Council District,Mapped Location
0,T2020016213,Building Residential - New,Single Family Residence,10216006100,03/11/2020,NaN,NaN,748 DARDEN PL,NASHVILLE,TN,37205,LOT 168 SEC 9 PT 2 HILLWOOD EST,Kingdom Builders of Tennesse,CARN,CAA01R301,3781725,New Single family dwelling. REJECTED: APPLICA...,23.0,"748 DARDEN PL\nNASHVILLE, TN 37205\n(36.125944..."
1,T2019073204,Building Moving Permit,Moving Permit - Residential,04600002700,12/02/2019,NaN,2500.0,4836 BULL RUN RD,ASHLAND CITY,TN,37015,N OF BULL RUN RD W OF OLD HICKORY BLVD,CLAYTON HOMES #054,CAMV,CAZ09A001,3736813,Move existing mobile home from property out of...,1.0,"4836 BULL RUN RD\nASHLAND CITY, TN 37015\n(36...."


In [3]:
df1_applic.shape

(3106, 19)

In [4]:
# Building Dept. Permits Issued
# low_memory = False was added to remove a low-memory warning. Doing this prevents the
# system from trying to assign dtypes until after the full file has been read
# Resource: https://tinyurl.com/stackoverflow-low-memory

df2_issued = pd.read_csv('../data/raw/Building_Permits_Issued_2020_06_05.csv'
                        , low_memory=False
                        )
df2_issued.head(2)

,Permit #,Permit Type Description,Permit Subtype Description,Parcel,Date Entered,Date Issued,Construction Cost,Address,City,State,ZIP,Subdivision / Lot,Contact,Permit Type,Permit Subtype,IVR Tracking #,Purpose,Council District,Census Tract,Mapped Location
0,2019070460,Building Residential - New,Single Family Residence,058100C04900CO,11/18/2019,12/09/2019,270585.0,1037 LAWSONS RIDGE DR,NASHVILLE,TN,37218,LOT 49 CARRINGTON PLACE PH 5,CELEBRATION HOMES LLC,CARN,CAA01R301,3733056,To construct a single family residence of 2402...,1.0,37010105.0,"1037 LAWSONS RIDGE DR\nNASHVILLE, TN 37218"
1,2020016259,Building Residential - Rehab,Single Family Residence,160150A07000CO,03/12/2020,03/12/2020,12000.0,210 HEARTHSTONE MANOR LN,BRENTWOOD,TN,37027,UNIT 70 HEARTHSTONE MANOR CONDOMINIUM PHASE 4,ACCESS & MOBILITY INC,CARR,CAA01R301,3781961,to install a new elevator/platform lift from g...,4.0,37018803.0,"210 HEARTHSTONE MANOR LN\nBRENTWOOD, TN 37027\..."


In [5]:
df2_issued.shape

(33909, 20)

## 1. Column name cleanup

In [6]:
# PERMIT APPLICATIONS

df1_applic.columns

Index(['Permit #', 'Permit Type Description', 'Permit Subtype Description',
       'Parcel', 'Date Entered', 'Date Issued', 'Construction Cost', 'Address',
       'City', 'State', 'ZIP', 'Subdivision / Lot', 'Contact', 'Permit Type',
       'Permit Subtype', 'IVR Tracking #', 'Purpose', 'Council District',
       'Mapped Location'],
      dtype='object')

In [7]:
df1_applic.columns = (df1_applic.columns
                      .str.replace(" ", "_")
                      .str.replace("/", "_")
                      .str.replace("Description", "descr")
                      .str.replace("#", "number")
                      .str.replace("___", "_")
                      .str.lower())
df1_applic.head(2)

,permit_number,permit_type_descr,permit_subtype_descr,parcel,date_entered,date_issued,construction_cost,address,city,state,zip,subdivision_lot,contact,permit_type,permit_subtype,ivr_tracking_number,purpose,council_district,mapped_location
0,T2020016213,Building Residential - New,Single Family Residence,10216006100,03/11/2020,NaN,NaN,748 DARDEN PL,NASHVILLE,TN,37205,LOT 168 SEC 9 PT 2 HILLWOOD EST,Kingdom Builders of Tennesse,CARN,CAA01R301,3781725,New Single family dwelling. REJECTED: APPLICA...,23.0,"748 DARDEN PL\nNASHVILLE, TN 37205\n(36.125944..."
1,T2019073204,Building Moving Permit,Moving Permit - Residential,04600002700,12/02/2019,NaN,2500.0,4836 BULL RUN RD,ASHLAND CITY,TN,37015,N OF BULL RUN RD W OF OLD HICKORY BLVD,CLAYTON HOMES #054,CAMV,CAZ09A001,3736813,Move existing mobile home from property out of...,1.0,"4836 BULL RUN RD\nASHLAND CITY, TN 37015\n(36...."


In [8]:
# PERMITS ISSUED

df2_issued.columns = (df2_issued.columns
                      .str.replace(" ", "_")
                      .str.replace("/", "_")
                      .str.replace("Description", "descr")
                      .str.replace("#", "number")
                      .str.replace("___", "_")
                      .str.lower())
df2_issued.head(2)

,permit_number,permit_type_descr,permit_subtype_descr,parcel,date_entered,date_issued,construction_cost,address,city,state,zip,subdivision_lot,contact,permit_type,permit_subtype,ivr_tracking_number,purpose,council_district,census_tract,mapped_location
0,2019070460,Building Residential - New,Single Family Residence,058100C04900CO,11/18/2019,12/09/2019,270585.0,1037 LAWSONS RIDGE DR,NASHVILLE,TN,37218,LOT 49 CARRINGTON PLACE PH 5,CELEBRATION HOMES LLC,CARN,CAA01R301,3733056,To construct a single family residence of 2402...,1.0,37010105.0,"1037 LAWSONS RIDGE DR\nNASHVILLE, TN 37218"
1,2020016259,Building Residential - Rehab,Single Family Residence,160150A07000CO,03/12/2020,03/12/2020,12000.0,210 HEARTHSTONE MANOR LN,BRENTWOOD,TN,37027,UNIT 70 HEARTHSTONE MANOR CONDOMINIUM PHASE 4,ACCESS & MOBILITY INC,CARR,CAA01R301,3781961,to install a new elevator/platform lift from g...,4.0,37018803.0,"210 HEARTHSTONE MANOR LN\nBRENTWOOD, TN 37027\..."


## 2a. Data types cleanup - Permit Applications

In [9]:
# PERMIT APPLICATIONS
# Confirming dtypes
# NEED TO CHANGE: date_entered should be datetime fields

df1_applic.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3106 entries, 0 to 3105
Data columns (total 19 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   permit_number         3106 non-null   object 
 1   permit_type_descr     3106 non-null   object 
 2   permit_subtype_descr  3106 non-null   object 
 3   parcel                3106 non-null   object 
 4   date_entered          3106 non-null   object 
 5   date_issued           0 non-null      float64
 6   construction_cost     1651 non-null   float64
 7   address               3106 non-null   object 
 8   city                  3106 non-null   object 
 9   state                 3106 non-null   object 
 10  zip                   3106 non-null   int64  
 11  subdivision_lot       3105 non-null   object 
 12  contact               3105 non-null   object 
 13  permit_type           3106 non-null   object 
 14  permit_subtype        3106 non-null   object 
 15  ivr_tracking_number  

In [10]:
# Convert date_entered to datetime 
#  (tried using .dt.date to keep date, only, but that changed it back into object dtype)
# .dt.date causes only the date to show

df1_applic.date_entered = pd.to_datetime(df1_applic.date_entered)
df1_applic.date_entered.head(2)

0   2020-03-11
1   2019-12-02
Name: date_entered, dtype: datetime64[ns]

In [11]:
# Double-checking min/max dates in this df

print(df1_applic.date_entered.min())
print(df1_applic.date_entered.max())

2017-06-01 00:00:00
2020-06-04 00:00:00


## 2b. Data types cleanup - Permits Issued

In [12]:
# PERMITS ISSUED
# Confirming dtypes
# NEED TO CHANGE: date_entered should be datetime field

df2_issued.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 33909 entries, 0 to 33908
Data columns (total 20 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   permit_number         33909 non-null  object 
 1   permit_type_descr     33909 non-null  object 
 2   permit_subtype_descr  33909 non-null  object 
 3   parcel                33909 non-null  object 
 4   date_entered          33909 non-null  object 
 5   date_issued           33909 non-null  object 
 6   construction_cost     33899 non-null  float64
 7   address               33909 non-null  object 
 8   city                  33909 non-null  object 
 9   state                 33909 non-null  object 
 10  zip                   33909 non-null  int64  
 11  subdivision_lot       33909 non-null  object 
 12  contact               33908 non-null  object 
 13  permit_type           33909 non-null  object 
 14  permit_subtype        33909 non-null  object 
 15  ivr_tracking_number

In [13]:
# Convert date_entered to datetime

df2_issued.date_entered = pd.to_datetime(df2_issued.date_entered)
df2_issued.date_entered.head(2)

0   2019-11-18
1   2020-03-12
Name: date_entered, dtype: datetime64[ns]

In [14]:
# Convert date_issued to datetime

df2_issued.date_issued = pd.to_datetime(df2_issued.date_issued)
df2_issued.date_issued.head(2)

0   2019-12-09
1   2020-03-12
Name: date_issued, dtype: datetime64[ns]

In [15]:
# Double-checking min/max dates in this df

print(df2_issued.date_issued.min())
print(df2_issued.date_issued.max())

2017-06-01 00:00:00
2020-06-04 00:00:00


## 3. New column: 'category' (Residential, Commercial, Other)

In [16]:
# Building Permit Applications df
# Stackoverflow resource: https://stackoverflow.com/questions/36653419/str-contains-to-create-new-column-in-pandas-dataframe

# Set a default value for new category column
df1_applic['category'] = 'Other'

# Assign Commercial, Residential based on permit_type AND permit_subtype, to capture the most possible of each one
# Doing BOTH type and sub_type increased Commercial category the most
df1_applic.loc[df1_applic['permit_type_descr'].str.contains('Commercial'), 'category'] = 'Commercial'
df1_applic.loc[df1_applic['permit_subtype_descr'].str.contains('Commercial'), 'category'] = 'Commercial'

df1_applic.loc[df1_applic['permit_type_descr'].str.contains('Residential'), 'category'] = 'Residential'
df1_applic.loc[df1_applic['permit_subtype_descr'].str.contains('Residential'), 'category'] = 'Residential'

print(df1_applic['category'].value_counts())
print(df1_applic.shape)

Residential    2204
Commercial      458
Other           444
Name: category, dtype: int64
(3106, 20)


In [17]:
# Building Permits Issued df
# Stackoverflow resource: https://stackoverflow.com/questions/36653419/str-contains-to-create-new-column-in-pandas-dataframe

# Set a default value for new category column
df2_issued['category'] = 'Other'

# Assign Commercial, Residential based on permit_type AND permit_subtype, to capture the most possible of each one
# Doing BOTH type and sub_type increased Commercial category the most
df2_issued.loc[df2_issued['permit_type_descr'].str.contains('Commercial'), 'category'] = 'Commercial'
df2_issued.loc[df2_issued['permit_subtype_descr'].str.contains('Commercial'), 'category'] = 'Commercial'

df2_issued.loc[df2_issued['permit_type_descr'].str.contains('Residential'), 'category'] = 'Residential'
df2_issued.loc[df2_issued['permit_subtype_descr'].str.contains('Residential'), 'category'] = 'Residential'

print(df2_issued['category'].value_counts())
print(df2_issued.shape)

Residential    22893
Commercial      7029
Other           3987
Name: category, dtype: int64
(33909, 21)


## 4. New column: 'type_subtype' concatenation

In [18]:
# BLDG PERMIT APPLICATIONS
# For use in Tableau visualization

df1_applic['type_subtype'] = ("TYPE: "
                              + df1_applic['permit_type_descr']
                              + " SUBTYPE: " 
                              + df1_applic['permit_subtype_descr']
                             )
df1_applic.head(1)     

,permit_number,permit_type_descr,permit_subtype_descr,parcel,date_entered,date_issued,construction_cost,address,city,state,...,subdivision_lot,contact,permit_type,permit_subtype,ivr_tracking_number,purpose,council_district,mapped_location,category,type_subtype
0,T2020016213,Building Residential - New,Single Family Residence,10216006100,2020-03-11,NaN,NaN,748 DARDEN PL,NASHVILLE,TN,...,LOT 168 SEC 9 PT 2 HILLWOOD EST,Kingdom Builders of Tennesse,CARN,CAA01R301,3781725,New Single family dwelling. REJECTED: APPLICA...,23.0,"748 DARDEN PL\nNASHVILLE, TN 37205\n(36.125944...",Residential,TYPE: Building Residential - New SUBTYPE: Sing...


In [19]:
# BLDG PERMITS ISSUED

df2_issued['type_subtype'] = ("TYPE: "
                              + df2_issued['permit_type_descr'] 
                              + " SUBTYPE: "
                              + df2_issued['permit_subtype_descr']
                             )
df2_issued.head(1)

,permit_number,permit_type_descr,permit_subtype_descr,parcel,date_entered,date_issued,construction_cost,address,city,state,...,contact,permit_type,permit_subtype,ivr_tracking_number,purpose,council_district,census_tract,mapped_location,category,type_subtype
0,2019070460,Building Residential - New,Single Family Residence,058100C04900CO,2019-11-18,2019-12-09,270585.0,1037 LAWSONS RIDGE DR,NASHVILLE,TN,...,CELEBRATION HOMES LLC,CARN,CAA01R301,3733056,To construct a single family residence of 2402...,1.0,37010105.0,"1037 LAWSONS RIDGE DR\nNASHVILLE, TN 37218",Residential,TYPE: Building Residential - New SUBTYPE: Sing...


## 5. Drop addresses with "0" house number

In [20]:
# Finding addresses that start with '0' house number

#df1_applic.loc[df1_applic['address'].str.startswith('0')]

# RESULT: 10 rows

In [21]:
# Building Permit Applications - Keeping all rows that DON'T have "0" for house number
# Original dataset 3106 rows. 
# After change, should be 3106 - 10 = 3096 rows  CONFIRMED

df1_applic = df1_applic[~df1_applic.address.str.startswith('0')]  # tilda means take everything EXECPT
df1_applic.shape 

(3096, 21)

In [22]:
# Finding addresses that start with '0' house number

#df2_issued.loc[df2_issued['address'].str.startswith('0')]

# RESULT: 40 rows

In [23]:
# Building Permits Isued - Keeping all rows that DON'T have "0" for house number
# Original dataset 3106 rows. After change, should be 33909 - 40 =  33869

df2_issued = df2_issued[~df2_issued.address.str.startswith('0')]  # tilda means take everything EXECPT
df2_issued.shape 

(33869, 22)

## 6. Pull lat/lon from mapped_location

In [24]:
# Building Permit Applications

df1_applic.mapped_location.unique()

array(['748 DARDEN PL\nNASHVILLE, TN 37205\n(36.125944, -86.879062)',
       '4836 BULL RUN RD\nASHLAND CITY, TN 37015\n(36.242681, -86.929594)',
       '4119 MURFREESBORO PIKE\nANTIOCH, TN 37013\n(36.032211, -86.594799)',
       ...,
       '6680 CHARLOTTE PIKE B-5\nNASHVILLE, TN 37209\n(36.136609, -86.883701)',
       '3805 CHARLOTTE AVE\nNASHVILLE, TN 37209\n(36.152561, -86.831473)',
       '5610A GRANNY WHITE PIKE\nBRENTWOOD, TN 37027\n(36.046438, -86.815953)'],
      dtype=object)

In [25]:
# To pull out lat/lng from mapped_location using regex / Regular Expression
# RegEx link:  https://regex101.com/r/cAI6sh/1

pattern = re.compile(r'.*\((\d*\S\d*)\S\s(\S\d*\S\d*)\)', flags = re.MULTILINE)

def extract_lat_lon(map_loc):
    '''To extract lat/lon from mapped_location
    in both bldg permit files'''
    try:
        lat_lon_match = pattern.search(map_loc)
        lat = float(lat_lon_match.group(1))
        lon = float(lat_lon_match.group(2))
        return(lat, lon)
    except:
        return(np.NaN, np.NaN)

In [26]:
# BLDG PERMIT APPLICATIONS
# Create list comprehension

lat_lon = [extract_lat_lon(map_loc) for map_loc in df1_applic.mapped_location]  #list comprehension

In [27]:
# Extract lat

df1_applic['lat'] = [lat for lat, lon in lat_lon]

In [28]:
# Extract lon

df1_applic['lon'] = [lon for lat, lon in lat_lon]

In [29]:
df1_applic.head(2)

,permit_number,permit_type_descr,permit_subtype_descr,parcel,date_entered,date_issued,construction_cost,address,city,state,...,permit_type,permit_subtype,ivr_tracking_number,purpose,council_district,mapped_location,category,type_subtype,lat,lon
0,T2020016213,Building Residential - New,Single Family Residence,10216006100,2020-03-11,NaN,NaN,748 DARDEN PL,NASHVILLE,TN,...,CARN,CAA01R301,3781725,New Single family dwelling. REJECTED: APPLICA...,23.0,"748 DARDEN PL\nNASHVILLE, TN 37205\n(36.125944...",Residential,TYPE: Building Residential - New SUBTYPE: Sing...,36.125944,-86.879062
1,T2019073204,Building Moving Permit,Moving Permit - Residential,04600002700,2019-12-02,NaN,2500.0,4836 BULL RUN RD,ASHLAND CITY,TN,...,CAMV,CAZ09A001,3736813,Move existing mobile home from property out of...,1.0,"4836 BULL RUN RD\nASHLAND CITY, TN 37015\n(36....",Residential,TYPE: Building Moving Permit SUBTYPE: Moving P...,36.242681,-86.929594


In [30]:
# BLDG PERMITS ISSUED
# Create list comprehension

lat_lon = [extract_lat_lon(map_loc) for map_loc in df2_issued.mapped_location]  #list comprehension

In [31]:
# Extract lat
# Warning is okay. 

df2_issued['lat'] = [lat for lat, lon in lat_lon]

In [32]:
# Extract lon
# Warning is okay. 

df2_issued['lon'] = [lon for lat, lon in lat_lon]

In [33]:
df2_issued.head(2)

,permit_number,permit_type_descr,permit_subtype_descr,parcel,date_entered,date_issued,construction_cost,address,city,state,...,permit_subtype,ivr_tracking_number,purpose,council_district,census_tract,mapped_location,category,type_subtype,lat,lon
0,2019070460,Building Residential - New,Single Family Residence,058100C04900CO,2019-11-18,2019-12-09,270585.0,1037 LAWSONS RIDGE DR,NASHVILLE,TN,...,CAA01R301,3733056,To construct a single family residence of 2402...,1.0,37010105.0,"1037 LAWSONS RIDGE DR\nNASHVILLE, TN 37218",Residential,TYPE: Building Residential - New SUBTYPE: Sing...,NaN,NaN
1,2020016259,Building Residential - Rehab,Single Family Residence,160150A07000CO,2020-03-12,2020-03-12,12000.0,210 HEARTHSTONE MANOR LN,BRENTWOOD,TN,...,CAA01R301,3781961,to install a new elevator/platform lift from g...,4.0,37018803.0,"210 HEARTHSTONE MANOR LN\nBRENTWOOD, TN 37027\...",Residential,TYPE: Building Residential - Rehab SUBTYPE: Si...,36.042219,-86.764816


## 7. Drop rows with null lat/lon

In [34]:
# BLDG PERMIT APPLICATIONS
# Find out how many of the mapped locations had only the address, not the lat/lon
# Number of rows missing lat & lon:  289

print(df1_applic.shape)
print(df1_applic.isnull().sum())

(3096, 23)
permit_number              0
permit_type_descr          0
permit_subtype_descr       0
parcel                     0
date_entered               0
date_issued             3096
construction_cost       1452
address                    0
city                       0
state                      0
zip                        0
subdivision_lot            1
contact                    1
permit_type                0
permit_subtype             0
ivr_tracking_number        0
purpose                   22
council_district           5
mapped_location            0
category                   0
type_subtype               0
lat                      289
lon                      289
dtype: int64


In [35]:
# Creating new BLDG PERMIT APPLICATIONS df that drops rows without lat/lon

# Double-checked a few things in these rows, one at a time
#print(df1_applic.shape)     #3096 before dropping null lat
# Should be 3096 - 289 = 2807 in new df  CONFIRMED

df1_applic = df1_applic.dropna(subset = ['lat'])  #dropping nulls in 'lat'
df1_applic.shape 

(2807, 23)

In [36]:
# BLDG PERMITS ISSUED
# Find out how many of the mapped locations had only the address, not the lat/lon
# Number of rows missing lat & lon:  3889

print(df2_issued.shape)
print(df2_issued.isnull().sum())

(33869, 24)
permit_number              0
permit_type_descr          0
permit_subtype_descr       0
parcel                     0
date_entered               0
date_issued                0
construction_cost         10
address                    0
city                       0
state                      0
zip                        0
subdivision_lot            0
contact                    1
permit_type                0
permit_subtype             0
ivr_tracking_number        0
purpose                  441
council_district          46
census_tract              43
mapped_location            0
category                   0
type_subtype               0
lat                     3889
lon                     3889
dtype: int64


In [37]:
# Creating new BLDG PERMITS ISSUED df that drops rows without lat/lon

# Double-checked a few things in these rows, one at a time
# Should be 33869 - 3889 = 29980 rows in new df  CONFIRMED

df2_issued = df2_issued.dropna(subset = ['lat'])  #dropping nulls in 'lat'
df2_issued.shape 

(29980, 24)

## 8. Sorting, before dropping duplicates
- Goal is to sort data in each df in a way that lends itself to the most appropriate dropping of rows that are "duplicates" (next step)

In [42]:
# BLDG PERMIT APPLICATIONS

# Sort by address, then type_subtype, then ivr_tracking_number.
# ivr_tracking_number is better than date becuse sometimes two entries have same date;
#      also, ivr_tracking_number is sequential (higher # = later date)
#      also, better than permt_number, which begins with 
#      either 'T' or 'D', which can throw off the sort.

df1_applic = df1_applic.sort_values(by = ['address'
                                          , 'type_subtype'
                                          , 'ivr_tracking_number']
                                   )
df1_applic = df1_applic.reset_index(drop = True)
print(type(df1_applic))
df1_applic.head(2)

<class 'pandas.core.frame.DataFrame'>


,permit_number,permit_type_descr,permit_subtype_descr,parcel,date_entered,date_issued,construction_cost,address,city,state,...,permit_type,permit_subtype,ivr_tracking_number,purpose,council_district,mapped_location,category,type_subtype,lat,lon
0,T2017061549,Building Commercial - Structural Frame,Hotel / Motel,09216039400,2017-09-21,NaN,1.0,1 MUSIC SQ W,NASHVILLE,TN,...,CACG,CAF14R102,3407957,structural frame permit only for proposed Virg...,19.0,"1 MUSIC SQ W\nNASHVILLE, TN 37203\n(36.151678,...",Commercial,TYPE: Building Commercial - Structural Frame S...,36.151678,-86.792944
1,T2017035948,Building Use & Occupancy,"Tents, Stages",09306406200,2017-06-01,NaN,2000.0,1 SYMPHONY PL,NASHVILLE,TN,...,CAUO,CAZ06A001,3371589,"(2) 40x80 tents, staked. No sidewall. Going up...",19.0,"1 SYMPHONY PL\nNASHVILLE, TN 37201\n(36.160351...",Other,"TYPE: Building Use & Occupancy SUBTYPE: Tents,...",36.160351,-86.775582


In [43]:
# BLDG PERMITS ISSUED

# Sort by address, then type_subtype, then ivr_tracking_number.
# ivr_tracking_number is better than date becuse sometimes two entries have same date;
#      also, ivr_tracking_number is sequential (higher # = later date)
#      also, better than permt_number, which begins with 
#      either 'T' or 'D', which can throw off the sort.

df2_issued = df2_issued.sort_values(by = ['address'
                                          , 'type_subtype'
                                          , 'ivr_tracking_number']
                                   )
df2_issued = df2_issued.reset_index(drop = True)
print(type(df2_issued))
df2_issued.head(2)

<class 'pandas.core.frame.DataFrame'>


,permit_number,permit_type_descr,permit_subtype_descr,parcel,date_entered,date_issued,construction_cost,address,city,state,...,permit_subtype,ivr_tracking_number,purpose,council_district,census_tract,mapped_location,category,type_subtype,lat,lon
0,2020011399,Building Commercial - Rehab,"General Office, Professional Services",09503002300,2020-02-21,2020-04-24,468115.0,1 BRIDGESTONE PARK,NASHVILLE,TN,...,CAD02B016,3761846,interior and exterior remodel of existing main...,15.0,37015100.0,"1 BRIDGESTONE PARK\nNASHVILLE, TN 37214\n(36.1...",Commercial,TYPE: Building Commercial - Rehab SUBTYPE: Gen...,36.166469,-86.690332
1,2018004314,Building Commercial - Rehab,"General Office, Professional Services",09503002300,2018-01-25,2018-03-16,130000.0,1 BRIDGESTONE PARK 100,NASHVILLE,TN,...,CAD02B016,3441067,Rehab existing first floor space for TENNESSE...,15.0,37015100.0,"1 BRIDGESTONE PARK 100\nNASHVILLE, TN 37214\n(...",Commercial,TYPE: Building Commercial - Rehab SUBTYPE: Gen...,36.166469,-86.690332


## 9. Drop duplicates:  Unable to do this. Will come back to it after internal walkthrough.
- Duplicates are defined as:
    - a row that has exact match on address, type_subtype, and that occurs within 90 days (~3 months) of the most recent occurance. 
    - Adding in the date limitation keeps in historical records for addresses that have more than one permit of the same type within a 3 year period; but eliminates those that are reissuances for thing like changing a contractor, or getting a new permit because the work hadn't been completed yet.
    


In [40]:
# BLDG PERMIT APPLICATIONS
# REFERENCE:  StackOverflow:
#https://stackoverflow.com/questions/47341275/drop-duplicate-rows-from-a-pandas-dataframe-whose-timestamps-are-within-a-specif

# Remove duplicates for those with same ADDRESS, and TYPE_SUBTYPE and that occur
#      within 90 DAYS of each other (keeping the latest entry)
# Drop based on [1] address, [2] type_subtype.

# TO CONFIRM: check 1 Titans Way, tents & stages, 3397152 should be gone, 3409055 should stay. CONFIRMED

# -----------------------------------------------------
# INITIAL TEST before saving to df -- found 168 duplicates to drop -- BUT NOT SURE THAT'S CORRECT 6/20/2020
# -----------------------------------------------------

#list_col = ['address', 'type_subtype']

#def drop_dups(x):
#    return x[list_col].duplicated & x.date_entered.diff().dt.days.lt(90)

#df1_applic.sort_values(list_col)
#df1_applic[~df1_applic.groupby(list_col).apply(drop_dups).values]



## 10. Convert to geodataframe

In [44]:
# BLDG PERMIT APPLICATIONS
# Convert df into gdf: https://gis.stackexchange.com/questions/147156/making-shapefile-from-pandas-dataframe

# STEP ONE: Create a new geometry column in your dataframe which combines the lat and lon values 
#           into a shapely Point() object. 

df1_applic['geometry'] = df1_applic.apply(lambda x: Point((float(x.lon)
                                                           , float(x.lat)))
                                          , axis = 1)

In [46]:
# STEP TWO: Convert the pandas DataFrame into a GeoDataFrame

df1_applic_geo = gpd.GeoDataFrame(df1_applic
                                  , geometry = 'geometry'
                                 )
type(df1_applic_geo)

geopandas.geodataframe.GeoDataFrame

In [52]:
df1_applic_geo.crs = 'EPSG:4326'
print(df1_applic_geo.crs)

EPSG:4326


In [49]:
# STEP ONE: Create a new geometry column in your dataframe which combines the lat and lon values 
#           into a shapely Point() object. 

df2_issued['geometry'] = df2_issued.apply(lambda x: Point((float(x.lon)
                                                               , float(x.lat)))
                                              , axis = 1)

In [50]:
# STEP TWO: Convert the pandas DataFrame into a GeoDataFrame

df2_issued_geo = gpd.GeoDataFrame(df2_issued
                                  , geometry = 'geometry'
                                 )
type(df1_applic_geo)

geopandas.geodataframe.GeoDataFrame

In [53]:
df2_issued_geo.crs = 'EPSG:4326'
print(df2_issued_geo.crs)

EPSG:4326


## 11. Keep only addresses within Davidson County

## 12. Add Neighborhood Association names

## 13. Merge

## 14. Export

### *To change row truncation, as needed*

In [41]:
# To get full results that aren't truncated, set to high number (default is 50):
pd.options.display.max_rows = 3000
print(pd.options.display.max_rows)

3000
